<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 6.0MB/s 
     |████████████████████████████████| 378kB 46.2MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=842394 sha256=88360f230ecb63ab6b036debfd1fe0f5a598341d4eabb8a6c7b1564b73de4c7a
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [0]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Check out [this issue](https://github.com/hanxiao/bert-as-service/issues/380) and "make sure Colab is using Tensorflow 1.x, because bert-serving-start doesn't currently work with TF 2.1 and nohup hides the output of the command failing"

In [0]:
# import tensorflow as tf
# print (tf.__version__)

In [3]:
!pip install -U bert-serving-server[http]
!pip install bert-serving-client  # client, independent of `bert-serving-server`

     |████████████████████████████████| 71kB 2.2MB/s 
     |████████████████████████████████| 1.1MB 6.9MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=4a8d18703baead10cbab730951fa95b23aa992e7870adbe7d782575ea07f4d46
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
  Created wheel for flask-compress: filename=Flask_Compress-1.4.0-cp36-none-any.whl size=3712 sha256=4febc4ca62f3529cb88fbeee77f77ea36b79a81db59db172083405dae82e63c5
  Stored in directory: /root/.cache/pip/wheels/96/32/88/a1f6d9dd3c29570ab3a8acc0d556b3b20abcf3c623c868ce0a
Successfully built GPUtil flask-compress
  Found existing installation: pyzmq 17.0.0
    Uninstalling pyzmq-17.0.0:
      Successfully uninstalled pyzmq-17.0.0


In [4]:
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip /content/multi_cased_L-12_H-768_A-12.zip

--2020-03-10 08:26:14--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M  81.2MB/s    in 7.5s    

2020-03-10 08:26:22 (83.7 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]

Archive:  /content/multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_

In [0]:
!nohup bert-serving-start -model_dir=./multi_cased_L-12_H-768_A-12 > out.file 2>&1 &

In [6]:
from bert_serving.client import BertClient
bc = BertClient()
encoded_test = bc.encode(['First do it', 'then do it right', 'then do it better'])
encoded_test

array([[ 0.49155793,  0.08796163,  0.08263294, ...,  1.0980438 ,
         0.411266  , -0.2539694 ],
       [-0.07413446, -0.22783582, -0.08978209, ...,  1.5093104 ,
         1.3512006 , -0.03158369],
       [-0.2657499 ,  0.1913579 , -0.35613576, ...,  1.3962182 ,
         1.4187475 ,  0.12651818]], dtype=float32)

In [0]:
encoded_test = bc.encode(['Вася любит Машу'])
encoded_test

In [0]:
# arg0, arg1, verb = 'маша петя любить'.split
def a0_a1_clause_maker(verb, arg0, arg1, arg0_case='nomn', arg1_case='accs', cap=True):

  '''Генерирует простые предложения вида 'arg0 любит arg1'.
  Параметры:
  verb              глагол в любой форме;
  arg0_case='nomn'  тег падежа подлежащего;
  arg1_case='accs'  тег падежа дополнения;
  arg0='Маша'       подлежащее;
  arg1='Петя'       дополнение
  cap=True        капитализация аргументов-имен собственных.
  Допустимые теги падежей лежат тут: http://opencorpora.org/dict.php?act=gram.'''

  inflected_arg0 = morph.parse(arg0)[0].inflect({'sing', 'nomn'}).word
  inflected_verb = morph.parse(verb)[0].inflect({'sing', '3per', 'pres', 'indc'}).word
  inflected_arg1 = morph.parse(arg1)[0].inflect({'sing', 'accs'}).word

  if (cap == True) and ('Name' in morph.parse(arg1)[0].tag):
    inflected_arg1 = inflected_arg1.capitalize()
  if (cap == True) and ('Name' in morph.parse(arg0)[0].tag):
    inflected_arg0 = inflected_arg0.capitalize()

  sentence = '{} {} {}'.format(inflected_arg0, inflected_verb, inflected_arg1)
  return sentence

# a0_a1_clause_maker('любить', 'Маша', 'Петя')
a0_a1_clause_maker('любить', 'Петя', 'Маша')


  

'Петя любит Машу'

In [0]:
# Пете нравится, что
def clause_wrapper(verb, arg0, arg1, 
                   main_subject=1, main_verb='нравиться',
                   arg0_case='nomn', arg1_case='accs', 
                   cap=True):

  '''Генерирует сложноподчинённые предложения вида 'arg нравится, что arg0 любит arg1'.
  Параметры:
  verb                    глагол-предикат подчинённой клаузы;
  arg0_case='nomn'        тег падежа подлежащего подчинённой клаузы;
  arg1_case='accs'        тег падежа дополнения подчинённой клаузы;
  main_subject            int, номер аргумента-подлежащего главной клаузы (по умолчанию дополнение подчинённой клаузы); 
  main_verb='нравиться',  глагол-предикат главной клаузы;
  arg0='Маша'             подлежащее подчинённой клаузы;
  arg1='Петя'             дополнение подчинённой клаузы;
  cap=True                капитализация аргументов-имен собственных.
  Допустимые теги падежей лежат тут: http://opencorpora.org/dict.php?act=gram.'''

  if main_subject:
    inflected_main_subject = morph.parse(arg1)[0].inflect({'sing', 'datv'}).word.capitalize()
  else:
    inflected_main_subject = morph.parse(arg0)[0].inflect({'sing', 'datv'}).word
  
  if (cap == True) and ('Name' in morph.parse(inflected_main_subject.)[0].tag):
    inflected_main_subject = inflected_main_subject.capitalize()


  inflected_main_verb = morph.parse(main_verb)[0].inflect({'sing', '3per', 'pres', 'indc'}).word
  subordinate_clause = a0_a1_clause_maker(verb, arg0, arg1, arg0_case, arg1_case, cap)
  main_clause = '{} {}'.format (inflected_main_subject, inflected_main_verb)
  sentence = '{}, что {}'.format(main_clause, subordinate_clause)

  return sentence

clause_wrapper('любить', 'Валя', 'Маша', 0)


'вале нравится, что Валя любит Машу'

In [0]:
# morph.parse('Пете')[0]
'Name' in morph.parse('Вале')[0].tag

False

In [0]:
# bc.server_status

# Определение удалённых сидов

In [0]:
def make_seed_embedding(seed_dict, model):
  '''
  Принимает на вход список, на выходе дает средний вектор всех слов из этого списка
  '''
  summ = 0
  # if model == skipgram_model:
  #   for verb in seed_dict:
  #     summ = summ + model[verb + '_V']
  #   vector = summ/len(seed_dict)
  if model == bc:
    for verb in seed_dict:
      summ = summ + bc.encode([verb])
    vector = summ/len(seed_dict)
  else:
    for verb in seed_dict:
      summ = summ + model.get_vector(verb)
    vector = summ/len(seed_dict)
  return vector

# seed_one_skipgram = make_seed_embedding(seed_one_dict, skipgram_model)
# seed_one_fasttext = make_seed_embedding(seed_one_dict, fasttext_model)

In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_distances

In [18]:
seed1pos = ['одобрять', 'хвалить', 'поощрять', 
            'обожать', 'восхищаться', 'восторгаться', 
            'нравиться', 'ценить', 'гордиться',
            'хвалить', 'нахваливать', 'превозносить']
seed1neg = ['порицать', 'осуждать', 'негодовать',
            'обвинять', 'наказывать', 
            'ненавидеть', 'убивать', 'разрушать']

avg1pos = make_seed_embedding(seed1pos, bc)
avg1neg = make_seed_embedding(seed1neg, bc)

cosine_distances(np.atleast_2d(avg1pos), np.atleast_2d(avg1pos))
# array([[0.49508154]], dtype=float32)
# cosine_distances(np.atleast_2d(avg2pos), np.atleast_2d(avg1neg))




array([[0.00822198]], dtype=float32)

In [0]:
from itertools import chain
from itertools import combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [0]:
neg_seeds_list = []
for i in powerset(seed1neg):
  if i:
    neg_seeds_list.append(list(i))
# neg_seeds_list

pos_seeds_list = []
for i in powerset(seed1pos):
  if i:
    pos_seeds_list.append(list(i))
# pos_seeds_list

In [0]:
pseeds_nseeds = np.zeros(shape=(len(pos_seeds_list),len(neg_seeds_list)))

In [42]:
pseeds_nseeds.shape

(4095, 255)

In [46]:


for i, ps in enumerate (pos_seeds_list):
  # print (i, ps)
  for j, ns in enumerate (neg_seeds_list):
    if pseeds_nseeds[i, j] == 0:
      seed1pos = pos_seeds_list[i]
      seed1neg = neg_seeds_list[j]
      avg1pos = make_seed_embedding(seed1pos, bc)
      avg1neg = make_seed_embedding(seed1neg, bc)
      distance = cosine_distances(np.atleast_2d(avg1pos), np.atleast_2d(avg1neg))
      pseeds_nseeds[i, j] = distance
      if (i%100 == 0) and (j==1):
        print(i)
%time
# np.unravel_index(np.argmin(pseeds_nseeds, axis=None), pseeds_nseeds.shape)

200
300
400
500


KeyboardInterrupt: ignored

In [0]:
# np.save('pseeds_nseeds_matrix', pseeds_nseeds)
pseeds_nseeds = np.load('pseeds_nseeds_matrix.npy')

In [0]:
pseeds_nseeds[538]

In [23]:
np.unravel_index(np.argmin(pseeds_nseeds, axis=None), pseeds_nseeds.shape)


(189, 248)

In [24]:
np.unravel_index(np.argmax(pseeds_nseeds, axis=None), pseeds_nseeds.shape)

(10, 1)

In [28]:
print(pos_seeds_list[189])
print(neg_seeds_list[248])
print(pseeds_nseeds[189, 248])

['поощрять', 'восхищаться', 'гордиться']
['порицать', 'осуждать', 'негодовать', 'обвинять', 'наказывать', 'убивать', 'разрушать']
0.0
